In [1]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
from scipy import ndimage as spn

In [2]:
# Current labels
scales = ['7', '12', '20', '36', '64', '122', 'ROI']
names_p = '/home/surchs/GDrive/PhD/Paper/Multiscale_Atlas/Labels/roi_label_scale_{}.csv'
proj_p = '/home/surchs/GDrive/PhD/Paper/Multiscale_Atlas/Labels/'
atlas_p = '/home/surchs/GDrive/PhD/Paper/Multiscale_Atlas/Partitions/scale_{}.nii.gz'

In [3]:
def find_symmetry(mask, affine):
    """
    Take the left side of the mask
    and flip it onto the right side 
    to check their overlap
    Bilaterality is continually coded:
    left: +1
    right: -1
    """
    # Get the midline
    origin = nib.affines.apply_affine(np.linalg.inv(affine), (0, 0, 0))
    midline = np.floor(origin[0]).astype(int)
    # Create the left and right section of the mask
    mask_left = np.zeros_like(mask)
    mask_right = np.zeros_like(mask)
    # Write the respective sections
    mask_left[:midline, ...] = mask[:midline, ...]
    mask_right[midline+1:, ...] = mask[midline+1:, ...]
    # Flip the left mask
    mask_flip = np.flip(mask_left, 0)
    # Compute dice
    dice = np.round(2*np.sum(mask_flip&mask_right)/(np.sum(mask_flip) + np.sum(mask_right)),2)
    # Check if the mask is bilateral
    bilateral = np.round((np.sum(mask_left) - np.sum(mask_right))/np.sum(mask),3)
    
    return dice, bilateral

In [4]:
def find_neighbour(roi, atlas):
    mask = atlas==roi
    dil_mask = spn.binary_dilation(mask)
    # Find guys
    neighbours = [n for n in np.unique(atlas[dil_mask]).astype(int) if not n==0 and not n==roi]
    return neighbours

In [5]:
# Get the atlas and backup names
atlas_i = nib.load(atlas_p.format(scales[0]))
affine = atlas_i.affine
atlas = {scale:nib.load(atlas_p.format(scale)).get_data() for scale in scales}
names = {scale:pd.read_csv(names_p.format(scale)) for scale in scales}

In [8]:
# Get stats on some of these guys
for scale in scales:
    # Get the ROIs
    template = atlas[scale]
    name = names[scale]
    rois = np.unique(template[template!=0])
    tl = list()
    for roi in rois:
        roi_mask = template==roi
        roi_lab = name[name['ROI']==roi]['roi_label'].values[0]
        roi_nam = name[name['ROI']==roi]['roi_name'].values[0]
        symmetry, laterality = find_symmetry(roi_mask, affine)
        com_vox = spn.measurements.center_of_mass(roi_mask)
        com_mni = np.round(nib.affines.apply_affine(affine, com_vox), 2)
        size = np.sum(roi_mask)
        neighbours = find_neighbour(roi, template)
        tl.append({'roi':roi.astype(int), 
                   'label':roi_lab, 
                   'name':roi_nam, 
                   'size':size, 
                   'symmetry':symmetry, 
                   'laterality':laterality,
                   'x':com_mni[0],
                   'y':com_mni[1],
                   'z':com_mni[2],
                   'neighbour':neighbours})
    table = pd.DataFrame(tl, columns=['roi', 'label', 'name', 'size', 
                                      'symmetry', 'laterality', 
                                      'x', 'y', 'z',
                                      'neighbour'])
    # Save the table
    table.to_csv(os.path.join(proj_p, 'roi_label_scale_{}.csv'.format(scale)), sep=';', index=False)